# Creating a Cutout with ERA5

In this example we download ERA5 data on-demand for a cutout we want to create.
(Atlite does also work with other datasources, but ERA5 is the easiest one to get started.)

This only works if you have in before

* Installed the Copernicus Climate Data Store `cdsapi` package
* Registered and setup your CDS API key as described [on their website here](https://cds.climate.copernicus.eu/api-how-to)

Import the package first:

In [1]:
import atlite

Disable urllib3 and cdsapi warnings.


We implement notifications in `atlite` using loggers from the `logging` library.

We recommend you always launch a logger to get information on what is going on.
For debugging, you can use the more verbose `level=logging.DEBUG`:

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

## Defining the Cutout extent

> This will not yet trigger any major operations.

A cutout is the basis for any of your work and calculations.

The `cutout` is created in the directory and file specified by the relative `path`
If a cutout at the given location already exists, then this command will simply load the cutout again.
If the cutout does not yet exist, it will specify the new to-be-created cutout.

In [3]:
cutout = atlite.Cutout(path="western-europe-2011-01.nc",
                       module="era5",
                       x=slice(-13.6913, 1.7712),
                       y=slice(49.9096, 60.8479),
                       time="2011-01"
                       )

INFO:atlite.cutout:Building new cutout western-europe-2011-01.nc


For creating the cutout, you need to specify

* The dataset to create the cutout with (`era5`)
* The time period it covers
* The longitude `x` and latitude `y` it stretches


Here we went with the `ERA5` dataset from ECMWF

```
module="era5"
```

Here we decided to provide the `time` period of the cutout as a string, because it is only a month.
You could have also specify it as a time range

```
slice("2011-01","2011-01")
```

The regional bounds (space the cutout stretches) where specified by the
```
x=slice(-13.6913, 1.7712) # Longitude
y=slice(49.9096, 60.8479) # Latitude
```

and describe a rectangle's edges.
In this case we drew a rectangle containing some parts of the atlantic ocean,
the Republic of Ireland and the UK.

## Preparing the Cutout

If the cutout does not yet exist or has some features which are not yet included
("The cutout is not prepared"), then we have to tell atlite to go ahead an do so.

No matter which dataset you use, this is where all the work actually happens.
This can be fast or take some or a lot of time and resources, among others depending on
your computer ressources and (for downloading e.g. ERA5 data) your internet connection.

In [4]:
cutout.prepare()

INFO:atlite.data:Calculating and writing with module era5:
INFO:atlite.datasets.era5:Downloading data for feature 'height' to /tmp/tmpgk8u6i16.
INFO:atlite.datasets.era5:Downloading data for feature 'wind' to /tmp/tmpgk8u6i16.


[########################################] | 100% Completed |  8.9s


INFO:atlite.datasets.era5:Downloading data for feature 'influx' to /tmp/tmpgk8u6i16.


[########################################] | 100% Completed | 11.1s


INFO:atlite.datasets.era5:Downloading data for feature 'temperature' to /tmp/tmpgk8u6i16.


...

INFO:atlite.datasets.era5:Downloading data for feature 'runoff' to /tmp/tmpgk8u6i16.


[########################################] | 100% Completed |  8.4s
[################                        ] | 40% Completed |  1.2s

/home/fabian/.miniconda3/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in true_divide
  return func(*args2)
/home/fabian/.miniconda3/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)


[########################################] | 100% Completed |  3.2s


<Cutout "western-europe-2011-01">
 x = -13.50 ⟷ 1.75, dx = 0.25
 y = 50.00 ⟷ 60.75, dy = 0.25
 time = 2011-01-01 ⟷ 2011-01-31, dt = H
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

Querying the cutout gives us some basic information on which data is contained
in it an can already be used.

In [5]:
cutout

<Cutout "western-europe-2011-01">
 x = -13.50 ⟷ 1.75, dx = 0.25
 y = 50.00 ⟷ 60.75, dy = 0.25
 time = 2011-01-01 ⟷ 2011-01-31, dt = H
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

We can access some information more specifically

In [6]:
cutout.prepared_features

module  feature    
era5    height                   height
        wind                    wnd100m
        wind                  roughness
        influx               influx_toa
        influx            influx_direct
        influx           influx_diffuse
        influx                   albedo
        temperature         temperature
        temperature    soil temperature
        runoff                   runoff
Name: variables, dtype: object

And the metadata used to create the cutout

In [7]:
cutout.data.attrs

{'module': 'era5',
 'prepared_features': ['height', 'wind', 'influx', 'temperature', 'runoff'],
 'chunks_time': 20}

We can access the underlying data (as a `xarray`).
This library implements lazy loading, so while we can preview the
data, it is usually only partially or fully loaded when we request
specific information or do operations on a whole feature parameter.
Information not loaded is indicated by the `...`:

In [8]:
cutout.data

<xarray.Dataset>
Dimensions:           (time: 744, x: 62, y: 44)
Coordinates:
  * x                 (x) float32 -13.5 -13.25 -13.0 -12.75 ... 1.25 1.5 1.75
  * y                 (y) float32 50.0 50.25 50.5 50.75 ... 60.25 60.5 60.75
    lat               (y) float32 dask.array<chunksize=(44,), meta=np.ndarray>
  * time              (time) datetime64[ns] 2011-01-01 ... 2011-01-31T23:00:00
    lon               (x) float32 dask.array<chunksize=(62,), meta=np.ndarray>
Data variables:
    height            (y, x) float32 dask.array<chunksize=(44, 62), meta=np.ndarray>
    wnd100m           (time, y, x) float32 dask.array<chunksize=(20, 44, 62), meta=np.ndarray>
    roughness         (time, y, x) float32 dask.array<chunksize=(20, 44, 62), meta=np.ndarray>
    influx_toa        (time, y, x) float32 dask.array<chunksize=(20, 44, 62), meta=np.ndarray>
    influx_direct     (time, y, x) float32 dask.array<chunksize=(20, 44, 62), meta=np.ndarray>
    influx_diffuse    (time, y, x) float32 dask.array<chunksize=(20, 44, 62), meta=np.ndarray>
    albedo            (time, y, x) float32 dask.array<chunksize=(20, 44, 62), meta=np.ndarray>
    temperature       (time, y, x) float32 dask.array<chunksize=(20, 44, 62), meta=np.ndarray>
    soil temperature  (time, y, x) float32 dask.array<chunksize=(20, 44, 62), meta=np.ndarray>
    runoff            (time, y, x) float32 dask.array<chunksize=(20, 44, 62), meta=np.ndarray>
Attributes:
    module:             era5
    prepared_features:  ['height', 'wind', 'influx', 'temperature', 'runoff']
    chunks_time:        20

If you have matplotlib installed, you can directly use the 
plotting functionality from `xarray` to plot features from
the cutout's data.

<div class="alert alert-info">

**Warning:** This will trigger `xarray` to load all the `height` data from disk into memory!

</div>